# Train scRF, a random forest-based classifier on a kidney 10k cells dataset

- Train a random forest classifier
- Format the classifier to use it with adverSCarial
- Run a max-change attack

In [1]:
library(randomForest)
library(dplyr)
library(adverSCarial)

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘dplyr’


The following object is masked from ‘package:randomForest’:

    combine


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
packageVersion("randomForest")

[1] ‘4.7.1.1’

In [3]:
c_basen = c("hgnc_axilla_10k", "hgnc_kidney_10k", "hgnc_liver_10k")
basen = c_basen[2]

In [4]:
df_train_boot <- read.table(paste0("data//v5/data//sc//",basen,"_train_balanced.txt"))
df_test <- read.table(paste0("data//v5/data//sc//",basen,"_test.txt"))

In [5]:
dim(df_train_boot)

[1]   600 32840

In [1]:
sqrt(32840)

[1] 181.2181

In [2]:
32840^(1/2)

[1] 181.2181

In [6]:
# Prepare data
x_train <- as.matrix(df_train_boot[, -which(names(df_train_boot) == "y")])
x_test <- as.matrix(df_test[, -which(names(df_test) == "y")])

In [7]:
y_train <- as.factor(df_train_boot$y)
y_test <- as.factor(df_test$y)

In [8]:
# Balanced training set
table(y_train)

y_train
  epithelial cell of proximal tubule                          kidney cell 
                                 100                                  100 
              kidney epithelial cell kidney loop of Henle epithelial cell 
                                 100                                  100 
                    mesenchymal cell                             podocyte 
                                 100                                  100 

Train model with default values

In [9]:
rf_model <- randomForest(x_train, y_train)

In [10]:
saveRDS(rf_model, paste0("repr_data/classifiers/scRF/",basen,"_random_forest_model.rds"))

In [11]:
rfModel <- readRDS(paste0("repr_data/classifiers/scRF/",basen,"_random_forest_model.rds"))

# Format the Classifier
To work with adverSCarial the classifier needs to be formated in a certain way.

In [12]:
scRF_classifier <- function(expr, clusters, target){
    library(randomForest)
    # Load the model
    rfModel <- readRDS(paste0("repr_data/classifiers/scRF/",basen,"_random_forest_model.rds"))
    # Predict with the model
    predictions <- predict(rfModel, expr, type="prob")
    if (sum(clusters == target) == 0 ){
        return( c("UNDETERMINED",1))
    }
    # Get the cell type with the highest odds
    cell_types <- apply(predictions[clusters == target,], 1, function(x){
        names(x[x == max(x)])[1]
    })
    table_cell_type <<- table(cell_types)
    str_class <- names(table_cell_type[order(table_cell_type, decreasing=T)][1])
    resSCtype <- list(
        # Cell type prediction for the cluster
        prediction=str_class,
        # Score of the predicted cell type
        odd=1,
        # Score for each cell type for each cell
        typePredictions=as.data.frame(t(predictions)),
        # Cell type for each cell
        cellTypes=cell_types)

    return(resSCtype)
}

In [13]:
df_pbmc_test <- df_test
expr_df <- df_pbmc_test[, -which(names(df_pbmc_test) == "y")]
clusters_df <- df_pbmc_test$y
names(clusters_df) <- rownames(df_pbmc_test)

Classification of a cluster.

In [14]:
myPreds <- scRF_classifier(expr_df, clusters_df, 'kidney cell')

In [15]:
# Cell type prediction for the cluster
myPreds$prediction

[1] "kidney cell"

In [16]:
# Score for each cell type for each cell
myPreds$typePredictions[1:5,1:5]

,4834STDY7002875_F16_KI_45P_CAGAATCTCGCCCTTA-1,4834STDY7002875_F16_KI_45P_CCTTCCCTCTGAGGGA-1,4834STDY7002875_F16_KI_45P_CGCGTTTAGGTGATTA-1,4834STDY7002875_F16_KI_45P_CTACCCAAGCTTATCG-1,4834STDY7002875_F16_KI_45P_TTCTCCTTCGAGAGCA-1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
epithelial cell of proximal tubule,0.096,0.046,0.046,0.072,0.016
kidney cell,0.214,0.216,0.160,0.230,0.214
kidney epithelial cell,0.208,0.212,0.176,0.420,0.510
kidney loop of Henle epithelial cell,0.140,0.050,0.498,0.110,0.088
mesenchymal cell,0.252,0.418,0.062,0.128,0.144


In [17]:
# Cell type for each cell
head(myPreds$cellTypes)

4834STDY7002875_F16_KI_45P_CAACCTCAGGAGCGAG-1 
                                "kidney cell" 
4834STDY7002875_F16_KI_45P_CCACCTATCGCATGGC-1 
                                   "podocyte" 
4834STDY7002875_F16_KI_45P_CTCGTCAAGGGCATGT-1 
                                "kidney cell" 
4834STDY7002875_F16_KI_45P_GAAGCAGCATCCTTGC-1 
                     "kidney epithelial cell" 
4834STDY7002875_F16_KI_45P_GACACGCGTCGCTTTC-1 
                                "kidney cell" 
4834STDY7002875_F16_KI_45P_GATCGATAGATGCCTT-1 
         "epithelial cell of proximal tubule"

## Check the prediction for each cell type

In [18]:
for ( myClust in unique(clusters_df)){
    myPreds <- scRF_classifier(expr_df, clusters_df, myClust)
    message("Cell type: ", myClust, ", prediction: ", myPreds$prediction)
}


Cell type: kidney epithelial cell, prediction: kidney epithelial cell

Cell type: kidney cell, prediction: kidney cell

Cell type: podocyte, prediction: podocyte

Cell type: epithelial cell of proximal tubule, prediction: epithelial cell of proximal tubule

Cell type: mesenchymal cell, prediction: mesenchymal cell

Cell type: kidney loop of Henle epithelial cell, prediction: kidney loop of Henle epithelial cell



## The function RF_classifier can be used as classifier with the adverSCarial package functions

# Run a max-change attack
Example of max-change attack with the positive-aberrant modification on the macrophage cluster.

In [19]:
start_time <- Sys.time()

In [20]:
myAttack <- advMaxChange(expr_df, clusters_df, 'kidney cell', scRF_classifier, advMethod = "perc99", maxSplitSize=1000)

predictWithNewValue data.frame data.frame

predictWithNewValue data.frame data.frame

predictWithNewValue data.frame data.frame

predictWithNewValue data.frame data.frame

result length: 32839



### Computation time

In [21]:
Sys.time() - start_time

Time difference of 1.458561 mins

In [26]:
length(myAttack@values)

[1] 32839

Check if the attack was successful.

In [23]:
mod_expr_df <- advModifications(expr_df, clusters=clusters_df, target='kidney cell', genes=myAttack@values, advMethod = "perc99")

In [24]:
mod_myPreds <- scRF_classifier(mod_expr_df, clusters_df, 'kidney cell')
mod_myPreds$prediction

[1] "kidney cell"

The cluster is still classified as "macrophage" after the modification of the 13714 genes.

In [25]:
sessionInfo()

R version 4.3.0 (2023-04-21)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 22.04.1 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/blas/libblas.so.3.10.0 
LAPACK: /usr/lib/x86_64-linux-gnu/lapack/liblapack.so.3.10.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=fr_FR.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=fr_FR.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=fr_FR.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=fr_FR.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Paris
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] adverSCarial_1.3.6   dplyr_1.1.2          randomForest_4.7-1.1

loaded via a namespace (and not attached):
 [1] Matrix_1.6-4          jsonlite_1.8.5        compiler_4.3.0       
 [4] crayon_1.5.2          t